In [ ]:
%%writefile .gitignore
.env
dev.env
.devcontainer/.env.runtime

mlruns/
mlflow_db/
mlruns_local/

node_modules/
frontend/node_modules/

archive/
.venv
uv.lock

test_iris.json
.env.template

# Railway CLI (never commit tokens)
.railway/config.json

archive/

In [ ]:
%%writefile backend/pyproject.toml
[project]
name = "react_fastapi_railway"
version = "0.1.0"
description = "Pytorch and Jax GPU docker container"
authors = [
  { name = "Geoffrey Hadfield" },
]
license = "MIT"
readme = "README.md"

# ─── Restrict to Python 3.10–3.12 ──────────────────────────────
requires-python = ">=3.10,<3.13"

dependencies = [
  "pandas>=1.2.0",
  "numpy>=1.20.0",
  "matplotlib>=3.4.0",
  "mlflow>=2.12.2",
  "mlflow-skinny>=2.12.2",
  "scikit-learn>=1.4.2",
  "pymc>=5.0.0",
  "arviz>=0.14.0",
  "statsmodels>=0.13.0",
  "jupyterlab>=3.0.0",
  "seaborn>=0.11.0",
  "tabulate>=0.9.0",
  "shap>=0.40.0",
  "xgboost>=1.5.0",
  "lightgbm>=3.3.0",
  "catboost>=1.2.8,<1.3.0",
  "scipy>=1.7.0",
  "shapash[report]>=2.3.0",
  "shapiq>=0.1.0",
  "explainerdashboard==0.5.1",
  "ipywidgets>=8.0.0",
  "nutpie>=0.7.1",   # new: nutpie backend for PyMC
  "numpyro>=0.18.0,<1.0.0",
  "jax==0.6.0",
  "jaxlib==0.6.0",
  "pytensor>=2.18.3",  # explicit version for CUDA support
  "aesara>=2.9.4",     # alternative backend option
  "tqdm>=4.67.0",
  "pyarrow>=12.0.0",
  "optuna>=3.0.0",
  "optuna-integration[mlflow]>=0.2.0",
  "omegaconf>=2.3.0,<2.4.0",
  "hydra-core>=1.3.2,<1.4.0",
  "fastapi>=0.104.0",
  "uvicorn[standard]>=0.24.0",
  "pydantic>=2.0.0",
  "pydantic-settings",
]

[project.optional-dependencies]
dev = [
  "pytest>=7.0.0",
  "black>=23.0.0",
  "isort>=5.0.0",
  "flake8>=5.0.0",
  "mypy>=1.0.0",
  "invoke>=2.2",
]

cuda = [
  "cupy-cuda12x>=12.0.0",  # For CUDA 12.x
]

[tool.pytensor]
# Default configuration for PyTensor
device = "cuda"          # Use CUDA by default if available
floatX = "float32"       # Use float32 by default for better CUDA performance
allow_gc = true          # Allow garbage collection
optimizer = "fast_run"   # Fast run optimization by default 


In [ ]:
%%writefile backend/requirements.txt
fastapi>=0.104.0
uvicorn[standard]>=0.24.0
python-dotenv>=1.0.0
pydantic>=2.0.0
pydantic-settings>=2.0.0
python-multipart>=0.0.6
httpx>=0.24.0
# Optional ML dependencies
numpy>=1.24.0
pandas>=2.1.0
scikit-learn>=1.3.0
# Optional MLflow integration
mlflow>=2.8.0 

In [ ]:
%%writefile backend/app/__init__.py
# FastAPI backend package 

In [ ]:
%%writefile backend/app/main.py
from fastapi import FastAPI, HTTPException, Request
from fastapi.middleware.cors import CORSMiddleware
from fastapi.staticfiles import StaticFiles
from fastapi.responses import FileResponse, JSONResponse
from pydantic import BaseModel
import os
from pathlib import Path
from typing import Dict, Any
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Create FastAPI app
app = FastAPI(
    title="FastAPI + React App",
    description="A full-stack application with FastAPI backend and React frontend",
    version="1.0.0",
    docs_url="/api/docs",
    redoc_url="/api/redoc",
    openapi_url="/api/openapi.json"
)

# CORS configuration
origins = [
    "http://localhost:3000",  # React development server (Create React App)
    "http://localhost:5173",  # React development server (Vite)
    "http://localhost:5174",  # Alternative Vite port
    "http://127.0.0.1:5173",  # Alternative localhost format
    "http://127.0.0.1:5174",  # Alternative localhost format
    "https://localhost:3000",
    "https://localhost:5173",
]

# Add Railway production URLs
railway_frontend_url = "https://react-frontend-production-2805.up.railway.app"
origins.append(railway_frontend_url)

# Add Railway domains and environment-specific origins
if os.getenv("RAILWAY_ENVIRONMENT"):
    # Add Railway deployment URL
    railway_url = os.getenv("RAILWAY_PUBLIC_DOMAIN")
    if railway_url:
        origins.extend([
            f"https://{railway_url}",
            f"http://{railway_url}"
        ])

# Add custom frontend URL from environment
frontend_url = os.getenv("FRONTEND_URL")
if frontend_url:
    origins.append(frontend_url)

# In development, allow all origins for easier debugging
if os.getenv("ENVIRONMENT") == "development" or not os.getenv("RAILWAY_ENVIRONMENT"):
    origins = ["*"]

# Log CORS configuration for debugging
logger.info(f"🔍 DEBUG: CORS origins configured: {origins}")
logger.info(f"🔍 DEBUG: RAILWAY_ENVIRONMENT: {os.getenv('RAILWAY_ENVIRONMENT')}")
logger.info(f"🔍 DEBUG: ENVIRONMENT: {os.getenv('ENVIRONMENT')}")
logger.info(f"🔍 DEBUG: FRONTEND_URL: {os.getenv('FRONTEND_URL')}")

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["GET", "POST", "PUT", "DELETE", "OPTIONS"],
    allow_headers=["*"],
    expose_headers=["*"]
)

# Pydantic models
class HealthResponse(BaseModel):
    status: str
    message: str
    version: str

class PredictionRequest(BaseModel):
    data: Dict[str, Any]

class PredictionResponse(BaseModel):
    prediction: Any
    confidence: float
    model_version: str

# API Routes
@app.get("/api/health", response_model=HealthResponse)
async def health_check():
    """Health check endpoint for Railway and monitoring"""
    return HealthResponse(
        status="healthy",
        message="FastAPI backend is running",
        version="1.0.0"
    )

@app.get("/api/ping")
async def ping():
    """Simple ping endpoint"""
    return {"message": "pong"}

@app.get("/api/hello")
async def hello():
    """Hello endpoint for React frontend to test API connection"""
    return {"message": "Hello from FastAPI backend!"}

@app.get("/api/info")
async def get_info():
    """Get application information"""
    return {
        "app_name": "FastAPI + React App",
        "version": "1.0.0",
        "environment": os.getenv("ENVIRONMENT", "production"),
        "railway_environment": os.getenv("RAILWAY_ENVIRONMENT"),
        "python_version": os.getenv("PYTHON_VERSION", "3.10")
    }

@app.post("/api/predict", response_model=PredictionResponse)
async def predict(request: PredictionRequest):
    """
    Example prediction endpoint
    Replace this with your actual ML model logic
    """
    try:
        logger.info("🔍 DEBUG: Received prediction request")
        logger.info(f"🔍 DEBUG: Request data: {request.data}")
        logger.info(f"🔍 DEBUG: Request type: {type(request)}")
        
        # Placeholder prediction logic
        # In a real app, you'd load your model and make predictions
        prediction_result = {
            "prediction": "sample_prediction",
            "confidence": 0.95,
            "model_version": "v1.0.0"
        }
        
        logger.info(f"🔍 DEBUG: Prediction result: {prediction_result}")
        
        response = PredictionResponse(**prediction_result)
        logger.info(f"🔍 DEBUG: Response object: {response}")
        logger.info(f"🔍 DEBUG: Response dict: {response.dict()}")
        
        return response
    
    except Exception as e:
        logger.error(f"🔍 DEBUG: Exception in predict endpoint: {str(e)}")
        logger.error(f"🔍 DEBUG: Exception type: {type(e)}")
        logger.error(f"🔍 DEBUG: Exception args: {e.args}")
        import traceback
        logger.error(f"🔍 DEBUG: Full traceback: {traceback.format_exc()}")
        raise HTTPException(status_code=500, detail=f"Prediction failed: {str(e)}")

# Static files serving for React app
# Check if frontend build exists
frontend_build_path = Path("frontend/dist")
if frontend_build_path.exists():
    # Mount static files
    app.mount("/static", StaticFiles(directory="frontend/dist/assets"), name="static")
    
    # Serve React app for all other routes (SPA routing)
    @app.get("/{full_path:path}")
    async def serve_react_app(full_path: str):
        """
        Serve React app for all non-API routes
        This enables client-side routing
        """
        # Don't serve React app for API routes
        if full_path.startswith("api/"):
            raise HTTPException(status_code=404, detail="API endpoint not found")
        
        # Serve index.html for all other routes
        index_file = frontend_build_path / "index.html"
        if index_file.exists():
            return FileResponse(index_file)
        else:
            raise HTTPException(status_code=404, detail="Frontend not found")
else:
    logger.warning("Frontend build directory not found. React app will not be served.")
    
    @app.get("/")
    async def root():
        """Root endpoint when frontend is not built"""
        return {
            "message": "FastAPI backend is running",
            "docs": "/api/docs",
            "frontend_status": "not_built"
        }

# Exception handlers
@app.exception_handler(404)
async def not_found_handler(request: Request, exc: HTTPException):
    """Custom 404 handler"""
    return JSONResponse(
        status_code=404,
        content={"error": "Not found", "path": str(request.url)}
    )

@app.exception_handler(500)
async def internal_error_handler(request: Request, exc: Exception):
    """Custom 500 handler"""
    logger.error(f"Internal server error: {exc}")
    return JSONResponse(
        status_code=500,
        content={"error": "Internal server error"}
    )

if __name__ == "__main__":
    import uvicorn
    port = int(os.getenv("PORT", 8000))
    host = "0.0.0.0"  # Railway requires binding to 0.0.0.0
    uvicorn.run(app, host=host, port=port) 